### Introduction
Servlet and the Servlet Container together interact with web-client via a request/reponse paradigm. So how does it all work?

1. Server receives request from a web client
2. Request is transferred to servlet container
3. Servlet container picks which servlet to invoke based on configuration
4. Servlet uses request and response object passed to it by the container and performs whatever logic it was programmed to
5. Servlet container then returns control back to the server

### Version History

| Version | Release Date | Platform           | Changes                         |
|---------|--------------|--------------------|---------------------------------|
| 1.0     | Dec 1996     | JSDK 1.0           |                                 |
| 2.0     | Dec 1997     | JSDK 2.0           |                                 |
| 2.1     | Nov 1998     | -                  |                                 |
| 2.2     | Nov 1999     | J2EE 1.2, J2SE 1.2 | Part of J2EE                    |
| 2.3     | Aug 2001     | J2EE 1.3, J2SE 1.2 | Filter added                    |
| 2.4     | Nov 2003     | J2EE 1.4, J2SE 1.3 |                                 |
| 2.5     | Sep 2005     | JEE 5, JSE 5       | Support annotations and Java 5  |
| 3.0     | Dec 2009     | JEE 6, JSE 6       | Async servlet, file upload, etc |
| 3.1     | May 2013     | JEE 7              | Non-blocking IO, etc            |
| 4.0     | Sep 2017     | JEE 8              | HTTP/2                          |


### Class Hierarchies
```
(I) Servlet 
        |
        +--- (AC) GenericServlet
                            |
                            +--- (AC) HttpServlet
```

### Servlet Lifecycle

![Lifecycle](https://i.imgur.com/qiGoPWz.png)

**How many servlet instance created?** One per servlet, however servlets implementing `SingleThreadModel`, servlet container may instantiate multiple instance to handle heavy request load. The use of the `SingleThreadModel` interface guarantees that only one thread at a time will execute in a given servlet instance’s service method.  

**Multithreading considerations:** Since multiple request can be handled by single servlet, it is important that the `service` method is thread safe. If the developer can't guarantee that then implement `SingleThreadModel` interface. Again synchronizing service method is highly discouraged since it will lead to loss of performance as requests to the servlet will be serialized.

### Servlet Shutdown
When the servlet container determines that a servlet should be removed fromk service, it calls the `destroy` method of the Servlet interface to allow the servlet to release any resources it is using and save any persistent state. 

Before the servlet container calls the `destroy` method, it must allow any threads that are currently running in the service method of the servlet to complete execution, or exceed a server-defined time limit. Once the `destroy` method is called on a servlet instance, the container may not route other requests to that instance of the servlet.

### Servlet Exceptions
Servlet may throw:
- ServletException
- UnavailableException: when servlet failed to initialize.

### Configuring Servlet
Servlet can be configured both using web.xml and annotations. A simple example including commonly used tags is listed below:

```xml
<web-app>
    <servlet>
        <servlet-name>InformationServlet</servlet-name>
        <servlet-class>servlets.InformationServlet</servlet-class>

        <!-- Obtain using servletConfig.getInitParameter("paramName") -->
        <init-param>
            <param-name>paramName</param-name>
            <param-value>paramValue</param-value>
        </init-param>

        <load-on-startup>0</load-on-startup>
    </servlet>
    
    <servlet-mapping>
        <servlet-name>InformationServlet</servlet-name>
        <url-pattern>/info</url-pattern>
    </servlet-mapping>
</web-app>
```

Using annotation form

In [ ]:
@WebServlet(urlPatterns = { "/info" }, description = "Servlet to get request information", loadOnStartup = 0, 
           initParams = {@WebInitParam(name = "paramName", value = "paramValue")})
public class InformationServlet extends HttpServlet {

}

### Exception Handling
We define Servlets (or JSPs) which will handle a particular exception object or HTTP error code as:

```xml
<!-- Servlet Mappings -->
<servlet-mapping>
   <servlet-name>ErrorHandlerServlet</servlet-name>
   <url-pattern>/error</url-pattern>
</servlet-mapping>

<!-- Http Error-Code related error pages -->
<error-page>
   <error-code>404</error-code>
   <location>/error</location>
</error-page>

<error-page>
   <error-code>403</error-code>
   <location>/error</location>
</error-page>

<!-- Exception-Type related error pages -->
<error-page>
   <exception-type>
      javax.servlet.ServletException
   </exception-type >
   <location>/error</location>
</error-page>
```

This error is forwarded to respective servlet as if `RequestDispatcher.forward` was invoked. There are several important information contained in the forwarded request. Important ones and repsective request attributes are:

| Information      | Attribute Name                     |
|------------------|------------------------------------|
| HTTP status code | javax.servlet.error.status_code    |
| Exception class  | javax.servlet.error.exception_type |
| Message          | javax.servlet.error.message        |
| Request URI      | javax.servlet.error.request_uri    |

We can programatically forward to error page using,

In [ ]:
response.sendError(ERROR_CODE);

### Asynchronous Servlet
Consider the scenario:  
![Async Servlet](images/async_servlet.png)

Using Async capability of serlvet, if a servlet or a filter reaches a potentially blocking operation when processing a request, it can assign the operation to an asynchronous execution context and return the thread associated with the request immediately to the container without generating a response. The blocking operation completes in the asynchronous execution context in a different thread, which can generate a response or dispatch the request to another servlet.  

Example:

In [ ]:
@WebServlet(urlPatterns={"/asyncservlet"}, asyncSupported=true) // asyncSupported value must be set to true
public class AsyncServlet extends HttpServlet {
   /* ... Same variables and init method as in SyncServlet ... */

   @Override
   public void doGet(HttpServletRequest request, 
                     HttpServletResponse response) {
      response.setContentType("text/html;charset=UTF-8");
      final AsyncContext acontext = request.startAsync();
      acontext.start(new Runnable() {
         public void run() {
            String param = acontext.getRequest().getParameter("param");
            String result = resource.process(param);
            HttpServletResponse response = acontext.getResponse();
            /* ... print to the response ... */
            acontext.complete();
   }
}